### This script will set up, simulate and gather all results for all cell rotations, for different model and/or field instantiations. The available settings are: 

**A:** Passive models with axons. Subthreshold simulation. Plane field 2.5 mm away from soma. <br />
**B:** Passive models without axons. Subthreshold simulation. Plane field 2.5 mm away from soma. <br />

**C:** Active models without axons. Subthreshold simulation. Plane field 2.5 mm away from soma. <br />
**D:** Active models without axons. Subthreshold simulation. Point field 50 um away from soma. <br />
**E:** Passive models without axons. Subthreshold simulation. Point field 50 um away from soma. <br />

**F:** Active models without axons. Suprathreshold simulation with calibrated DC injection. Plane field 2.5 mm away from soma. <br />
**G:** Active models without axons. Suprathreshold simulation with calibrated DC injection. Point field 50 um away from soma. <br />

In [1]:
Selected_Sweep = 'G'

simulations_in_parallel = 26
model_processing = 'csmc_allactive'
save_trace = False

Sub_amplitudes = [1,3,5,10,30,50,100,300,500,1000] # uA
Sub_frequencies =[1,3,5,8,10,28,30,50,100,140,300] # Hz
SuP_amplitudes = [0.0,10,100,500] # uA
SuP_amplitudes_P = [0.0,0.05,0.1,0.2] # uA
SuP_frequencies =[8,28,140] # Hz

if Selected_Sweep == 'A':
    model_file = '../Required_Files/Models/JSON Format/' + Selected_Sweep + '.json'
    swc_path = '../Required_Files/Models/SWCs connected With/'
    supra = False
    point = False
    Rotation_Steps = 30 # deg
    Only_Calibrated = False
    Hard = False
elif Selected_Sweep == 'B':
    model_file = '../Required_Files/Models/JSON Format/A.json'
    swc_path = '../Required_Files/Models/SWCs connected Without/'
    supra = False
    point = False
    Rotation_Steps = 30 # deg
    Only_Calibrated = False
    Hard = False
    save_trace = True
elif Selected_Sweep == 'C':
    model_file = False
    swc_path = '../Required_Files/Models/SWCs connected Without/'
    supra = False
    point = False
    Rotation_Steps = 30 # deg
    Only_Calibrated = False
    Hard = False
    Sub_amplitudes = [10,100,500] # uA
    save_trace = True
elif Selected_Sweep == 'D':
    model_file = False
    swc_path = '../Required_Files/Models/SWCs connected Without/'
    supra = False
    point = True
    Rotation_Steps = 10 # deg
    Only_Calibrated = False
    Hard = False
    Sub_amplitudes = [10,100,500] # uA
    Sub_frequencies =[1,8,28,140]
elif Selected_Sweep == 'E':
    model_file = '../Required_Files/Models/JSON Format/A.json'
    swc_path = '../Required_Files/Models/SWCs connected Without/'
    supra = False
    point = True
    Rotation_Steps = 10 # deg
    Only_Calibrated = False
    Hard = False
    Sub_amplitudes = [10,100,500] # uA
    Sub_frequencies =[1,8,28,140]
elif Selected_Sweep == 'F':
    model_file = False
    swc_path = '../Required_Files/Models/SWCs connected Without/'
    supra = True
    point = False
    Rotation_Steps = 30 # deg
    Only_Calibrated = True
    Hard = False
elif Selected_Sweep == 'G':
    model_file = False
    swc_path = '../Required_Files/Models/SWCs connected Without/'
    supra = True
    point = True
    Rotation_Steps = 10 # deg
    Only_Calibrated = True
    Hard = False

##### Set up directories

In [2]:
import shutil, os, json
import pandas as pd

morphfiles = os.listdir(swc_path)

inhpath = '../Required_files/Models/JSON Format/Inh Models/'
excpath = '../Required_files/Models/JSON Format/Exc Models/'
inhtypes = os.listdir(inhpath)
exctypes = os.listdir(excpath)

allmodels = []
for i in inhtypes:
    for j in os.listdir(inhpath + i):
        allmodels.append(inhpath + i + '/' + j + '/' + j + '_stage_1_hof_0.json')

for i in exctypes:
    for j in os.listdir(excpath + i):
        allmodels.append(excpath + i + '/' + j + '/' + j + '_stage_1_hof_0.json')

current_vals = pd.read_csv('../Required_files/Calibrated.csv')
current_vals = current_vals.sort_values('Target', ascending=True).reset_index(drop=True).reset_index()
current_vals = current_vals.set_index('cell_id')

simulation_no = 0
for i in morphfiles:
    if int(i.split('rmed_')[1].split('.sw')[0]) not in range(0,360,Rotation_Steps):
        continue
    if int(i.split('_')[2]) not in current_vals.index and Only_Calibrated:
        continue
    if supra:
        frequencies = SuP_frequencies
    else:
        frequencies = Sub_frequencies
    for f in frequencies:
        if supra:
            if point:
                amplitudes = SuP_amplitudes_P
            else:
                amplitudes = SuP_amplitudes
        else:
            amplitudes = Sub_amplitudes
        for c in amplitudes:
            full_path = os.getcwd().replace('\\','/') + './Simulations/' + Selected_Sweep + '/SIM_' + str(simulation_no)
            if os.path.exists(full_path):
                if os.path.isfile(full_path + '/Data.csv'):
                    simulation_no += 1
                    continue
                else:
                    shutil.rmtree(full_path)
            os.makedirs(full_path)
            shutil.copyfile(swc_path + i, full_path + '/morph.swc')
            shutil.copyfile('../Required_Files/Simulate.py', full_path + '/simulate.py')
            shutil.copyfile('../Required_Files/cell_functions.py', full_path + '/cell_functions.py')
            shutil.copyfile('../Required_Files/nrnmech.dll', full_path + '/nrnmech.dll')
            if model_file:
                shutil.copyfile(model_file, full_path + '/model.json')
            else:
                for mod in allmodels:
                    if i.split("transformed")[0] in mod:
                        inpa = mod
                        oupa = full_path + '/model.json'
                        with open(inpa, 'r') as infile:
                            parameters = json.load(infile)
                        del parameters['axon_morph']
                        if ('Without' not in swc_path) or Hard:
                            axonlist = []
                            gendict = parameters['genome']
                            for di in gendict:
                                if di['section']=='soma':
                                    axonlist.append(di.copy())
                            for gi in range(len(axonlist)):
                                axonlist[gi]['section']='axon'
                                gendict.append(axonlist[gi])
                            parameters['genome'] = gendict
                        elif ('Without' not in swc_path):
                            axonlist = []
                            gendict = parameters['genome']
                            for di in gendict:
                                if di['section']=='dend':
                                    axonlist.append(di.copy())
                            for gi in range(len(axonlist)):
                                axonlist[gi]['section']='axon'
                                gendict.append(axonlist[gi])
                            parameters['genome'] = gendict
                        json_object = json.dumps(parameters, indent=4)
                        with open(oupa, 'w') as outfile:
                            outfile.write(json_object)
                        break
            if supra:
                inj_curr = current_vals.loc[int(i.split('_')[2]),'Mean']
                inj_std = current_vals.loc[int(i.split('_')[2]),'StD']
                exp_rate = current_vals.loc[int(i.split('_')[2]),'PM']/2
            else:
                inj_curr = 0
                inj_std = 0
                exp_rate = 200
            fin = open(full_path + '/simulate.py', "rt")
            cnt = 10
            lines = []
            for line in fin:
                if cnt==10:
                    lines.append('base_dir = "' + full_path + '"\n')
                elif cnt==9:
                    lines.append('inj_curr = ' + str(inj_curr) + '\n')
                elif cnt==8:
                    lines.append('inj_std = ' + str(inj_std) + '\n')
                elif cnt==7:
                    lines.append('exp_rate = ' + str(exp_rate) + '\n')
                elif cnt==6:
                    lines.append('es_freq = ' + str(f) + '\n')
                elif cnt==5:
                    lines.append('es_ampl = ' + str(c) + '\n')
                elif cnt==4:
                    lines.append('mod_proc = "' + model_processing + '"\n')
                elif cnt==3:
                    if point:
                        lines.append('point = 1\n')
                    else:
                        lines.append('point = 0\n')
                elif cnt==2:
                    lines.append('run = 1\n')
                elif cnt==1:
                    if supra:
                        lines.append('returns = 1\n')
                    else:
                        lines.append('returns = 0\n')
                else:
                    lines.append(line)
                cnt -= 1
            fin.close()
            fout = open(full_path + '/simulate.py', "wt")
            for line in lines:
                fout.write(line)
            fout.close()

            simulation_no += 1
print (simulation_no)

12960


##### Simulate

In [ ]:
import subprocess as sbp
import os, shutil, time

total_simulations = simulation_no
print(total_simulations)
processes = []
process_names = []
active_count = 0
process_index = 0

start_time = time.time()
while (active_count > 0) or (process_index < total_simulations):
    active_processes = processes.copy()
    active_process_names = process_names.copy()
    processes = []
    process_names = []
    for i, sbprc in enumerate(active_processes):
        poll = sbprc.poll()
        if poll is None:
            processes.append(sbprc)
            process_names.append(active_process_names[i])
        else:
            active_count -= 1
            print ('Finished ' + active_process_names[i])
            if process_index >= total_simulations:
                print ('Active: ' + str(active_count) + ', Pending: ' + str(total_simulations-process_index))
    if (active_count < simulations_in_parallel) and (process_index < total_simulations):
        process_name = 'SIM_'+str(process_index)
        if not os.path.isfile(os.getcwd().replace('\\','/') + '/SIMULATIONS/' + Selected_Sweep + '/' + process_name + "/Data.csv"):
            cmnd = ["python", 
                    os.getcwd().replace('\\','/') + '/SIMULATIONS/' + Selected_Sweep + '/' + process_name + "/simulate.py"]
            p = sbp.Popen(cmnd)
            processes.append(p)
            process_names.append(process_name)
            active_count += 1
        process_index += 1
        print ('Active: ' + str(active_count) + ', Pending: ' + str(total_simulations-process_index))
    if active_count >= simulations_in_parallel:
        time.sleep(2)
    
queue_time = time.time() - start_time
print("Total time " + str(queue_time/3600) + " hours")

12960
Active: 1, Pending: 12959
Active: 2, Pending: 12958
Active: 3, Pending: 12957
Active: 4, Pending: 12956
Active: 5, Pending: 12955
Active: 6, Pending: 12954
Active: 7, Pending: 12953
Active: 8, Pending: 12952
Active: 9, Pending: 12951
Active: 10, Pending: 12950
Active: 11, Pending: 12949
Active: 12, Pending: 12948
Active: 13, Pending: 12947
Active: 14, Pending: 12946
Active: 15, Pending: 12945
Active: 16, Pending: 12944
Active: 17, Pending: 12943
Active: 18, Pending: 12942
Active: 19, Pending: 12941
Active: 20, Pending: 12940
Active: 21, Pending: 12939
Active: 22, Pending: 12938
Active: 23, Pending: 12937
Active: 24, Pending: 12936
Active: 25, Pending: 12935
Active: 26, Pending: 12934
Finished SIM_0
Finished SIM_1
Finished SIM_2
Finished SIM_3
Finished SIM_4
Finished SIM_5
Finished SIM_6
Finished SIM_7
Finished SIM_8
Finished SIM_9
Finished SIM_10
Finished SIM_11
Finished SIM_12
Finished SIM_13
Finished SIM_14
Finished SIM_15
Finished SIM_16
Finished SIM_17
Finished SIM_18
Finishe

Finished SIM_164
Active: 26, Pending: 12760
Finished SIM_174
Active: 26, Pending: 12759
Finished SIM_175
Active: 26, Pending: 12758
Finished SIM_177
Active: 26, Pending: 12757
Finished SIM_176
Active: 26, Pending: 12756
Finished SIM_178
Active: 26, Pending: 12755
Finished SIM_204
Active: 26, Pending: 12754
Finished SIM_205
Active: 26, Pending: 12753
Finished SIM_206
Active: 26, Pending: 12752
Finished SIM_207
Active: 26, Pending: 12751
Finished SIM_208
Active: 26, Pending: 12750
Finished SIM_209
Active: 26, Pending: 12749
Finished SIM_210
Active: 26, Pending: 12748
Finished SIM_211
Active: 26, Pending: 12747
Finished SIM_212
Active: 26, Pending: 12746
Finished SIM_213
Active: 26, Pending: 12745
Finished SIM_180
Active: 26, Pending: 12744
Finished SIM_214
Active: 26, Pending: 12743
Finished SIM_215
Active: 26, Pending: 12742
Finished SIM_181
Finished SIM_216
Active: 25, Pending: 12741
Active: 26, Pending: 12740
Finished SIM_217
Active: 26, Pending: 12739
Finished SIM_218
Finished SIM_21

Finished SIM_345
Active: 26, Pending: 12573
Finished SIM_385
Finished SIM_386
Active: 25, Pending: 12572
Active: 26, Pending: 12571
Finished SIM_387
Finished SIM_388
Active: 25, Pending: 12570
Active: 26, Pending: 12569
Finished SIM_389
Finished SIM_390
Active: 25, Pending: 12568
Active: 26, Pending: 12567
Finished SIM_391
Finished SIM_392
Active: 25, Pending: 12566
Active: 26, Pending: 12565
Finished SIM_360
Active: 26, Pending: 12564
Finished SIM_393
Finished SIM_394
Active: 25, Pending: 12563
Active: 26, Pending: 12562
Finished SIM_395
Active: 26, Pending: 12561
Finished SIM_396
Finished SIM_397
Active: 25, Pending: 12560
Active: 26, Pending: 12559
Finished SIM_398
Active: 26, Pending: 12558
Finished SIM_399
Finished SIM_400
Active: 25, Pending: 12557
Active: 26, Pending: 12556
Finished SIM_401
Active: 26, Pending: 12555
Finished SIM_402
Finished SIM_403
Active: 25, Pending: 12554
Active: 26, Pending: 12553
Finished SIM_404
Active: 26, Pending: 12552
Finished SIM_405
Finished SIM_40

Finished SIM_550
Active: 26, Pending: 12386
Finished SIM_551
Active: 26, Pending: 12385
Finished SIM_546
Active: 26, Pending: 12384
Finished SIM_552
Active: 26, Pending: 12383
Finished SIM_553
Active: 26, Pending: 12382
Finished SIM_555
Active: 26, Pending: 12381
Finished SIM_549
Active: 26, Pending: 12380
Finished SIM_556
Active: 26, Pending: 12379
Finished SIM_557
Active: 26, Pending: 12378
Finished SIM_558
Active: 26, Pending: 12377
Finished SIM_559
Active: 26, Pending: 12376
Finished SIM_547
Active: 26, Pending: 12375
Finished SIM_554
Active: 26, Pending: 12374
Finished SIM_560
Active: 26, Pending: 12373
Finished SIM_561
Active: 26, Pending: 12372
Finished SIM_562
Active: 26, Pending: 12371
Finished SIM_564
Active: 26, Pending: 12370
Finished SIM_565
Active: 26, Pending: 12369
Finished SIM_569
Active: 26, Pending: 12368
Finished SIM_563
Active: 26, Pending: 12367
Finished SIM_568
Active: 26, Pending: 12366
Finished SIM_566
Active: 26, Pending: 12365
Finished SIM_567
Active: 26, Pen

Create table

In [ ]:
import shutil, os, json, h5py
import pandas as pd
import numpy as np
import  scipy.stats as sps
import pycircstat

morphfiles = os.listdir(swc_path)
current_vals = pd.read_csv('../Required_files/Calibrated.csv')
current_vals = current_vals.sort_values('Target', ascending=True).reset_index(drop=True).reset_index()
current_vals = current_vals.set_index('cell_id')

def MMR (PhaseList):
    instances = np.array(np.round(PhaseList), dtype = int)
    bin_size = 36  
    gaussian = []
    for i in range(bin_size*2+1):
        gaussian.append(np.exp(-0.5*(i-bin_size)**2/(bin_size/4)**2)/(bin_size*np.sqrt(2*np.pi)/4))
    gaussian = np.array(gaussian)/gaussian[bin_size]
    waveshaped = []
    for i in instances:
        waveshape = np.zeros(360)
        indexes = np.arange(0,len(gaussian))
        indexes = indexes+i-bin_size
        for j in np.arange(0,len(gaussian)):
            if indexes[j]<0:
                indexes[j] +=360
            elif indexes[j]>359:
                indexes[j] -=360
        for j, ind in enumerate(indexes):
            waveshape[ind] = gaussian [j]
        waveshaped.append(waveshape)
    sumation = np.sum(waveshaped, axis = 0)
    sumation = sumation/np.max(sumation)
    return sumation, 1-np.mean(sumation)

all_results = []
all_traces = []
simulation_no = 0
for i in morphfiles:
    if int(i.split('rmed_')[1].split('.sw')[0]) not in range(0,360,Rotation_Steps):
        continue
    if int(i.split('_')[2]) not in current_vals.index and Only_Calibrated:
        continue
    if supra:
        frequencies = SuP_frequencies
    else:
        frequencies = Sub_frequencies
    for f in frequencies:
        if supra:
            if point:
                amplitudes = SuP_amplitudes_P
            else:
                amplitudes = SuP_amplitudes
        else:
            amplitudes = Sub_amplitudes
        for c in amplitudes:
            full_path = os.getcwd().replace('\\','/') + '/Simulations/' + Selected_Sweep + '/SIM_' + str(simulation_no)
            if supra:
                try:
                    full_path += '/Data.csv'
                    df = pd.read_csv(full_path)
                except:
                    if point==0:
                        print("File missing: "+full_path)    
                    simulation_no += 1
                    continue
                Peak_indices = np.array(df.iloc[:,0])
                ISIs = Peak_indices[1:]-Peak_indices[:-1]
                ISIs = 20000 / ISIs
                mean,std=sps.norm.fit(ISIs)

                Phases = Peak_indices % (20000/f)
                Phases = Phases * 360 * f /20000
                radians = np.asarray(Phases)*np.pi/180
                mean_phase_rad = pycircstat.descriptive.mean(np.array(radians))
                mean_phase_angle = mean_phase_rad*(180 / np.pi)
                mean_pvalue_z=pycircstat.rayleigh(np.array(radians))
                mean_vector_length = pycircstat.descriptive.vector_strength(np.array(radians))
        
                cd, mmr = MMR(Phases)
                Y, X = np.histogram(Phases, bins=[0, 24, 48, 72, 96, 120, 144, 168, 192, 216, 240, 264, 288, 312, 336, 360])
                Xp =(X[1:] + X[:-1]) / 2
                Xp = Xp * np.pi / 180
                normY = np.true_divide(Y, (np.max(Y)))
                thetas = []
                cnt = 0.0
                for cn in range(len(cd)):
                    thetas.append(cnt)
                    cnt = cnt + 2*np.pi/len(cd)
                all_results.append([i.split('_')[0],i.split('_')[2],i.split('_')[4].replace('.swc',''), f, c, 
                          mean_vector_length, mean_pvalue_z, mean_phase_angle, mmr, 
                          thetas, cd, Xp, normY, list(Peak_indices), list(ISIs), list(Phases)])
            else:
                try:
                    full_path += '/Data.csv'
                    data = pd.read_csv(full_path)
                except:
                    if point==0:
                        print("File missing: "+full_path)    
                    simulation_no += 1
                    continue
                row_res = [i.split('_')[0],i.split('_')[2],i.split('_')[4].replace('.swc',''),f,c]
                for r in data['0']:
                    row_res.append(r)
                if save_trace:
                    neuron_data_output_file = full_path.replace('/Data.csv','/Simulation/output/v_report.h5')
                    with h5py.File(neuron_data_output_file, "r") as fi:
                        report = fi['report']
                        single_neuron = report['single_neuron']
                        data=np.array(single_neuron['data'][()])[45000:55000]
                    fi.close()
                    all_traces.append([i.split('_')[0],i.split('_')[2],i.split('_')[4].replace('.swc',''),f,c,data])
                all_results.append(row_res)
            simulation_no += 1
if supra:
    gf = pd.DataFrame(all_results, columns = ['Cell_Type', 'Cell_ID', 'Cell_Rotation(deg)', 'ES_frequency(hz)',
                                              'ES_current(uA)', 'mean_vector_length', 'mean_pvalue_z', 
                                              'mean_phase_angle', 'MMR', 'MMR_X', 'MMR_Y', 'VL_X', 'VL_Y', 
                                              'Peak_indices', 'ISIs', 'Phases'])
    gf.to_pickle('../Results/Result_Tables/'+ Selected_Sweep +'.pkl')
else:
    df = pd.DataFrame(all_results)
    df.columns = ['Cell_Type', 'Cell_ID', 'Cell_Rotation(deg)', 'ES_frequency(hz)',
              'ES_current(uA)', 'Raw_Min(mV)', 'Raw_Max(mV)', 'Raw_Mean(mV)',
              'Raw_Median(mV)', 'Amp_Min(mV)', 'Amp_Max(mV)', 'Amp_Mean(mV)',
              'Amp_Median(mV)','Amp@40000(mV)', 'Phs_Min(deg)', 'Phs_Max(deg)', 
              'Phs_Mean(deg)', 'Phs_Median(deg)', 'Phs@30000(deg)', 'Phs@40000(deg)']
    df.to_csv('../Results/Result_Tables/'+ Selected_Sweep +'.csv', index = False)
    if save_trace:
        ff = pd.DataFrame(all_traces)
        ff.columns = ['Cell_Type', 'Cell_ID', 'Cell_Rotation(deg)', 'ES_frequency(hz)',
                  'ES_current(uA)', 'Trace(mV)']
        ff.to_pickle('../Results/Result_Tables/'+ Selected_Sweep +'_Trace.pkl')    

Erase simulation files

In [ ]:
import shutil, os

shutil.rmtree(os.getcwd().replace('\\','/') + '/Simulations/' + Selected_Sweep)